<a href="https://colab.research.google.com/github/thiagodsd/dicewin/blob/master/notebooks_annotations/revisao_e_snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

In [0]:
import numpy  as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

## 1. Classical Learning

### Supervised

#### Regression

#### Classification

#### SVM

IMPORTANTISSIMO: a `cross_val_score` retorna o score no **teste** sempre! Entao algo como

```python
scr_f1 = cross_val_score(SVC(kernel='linear', C=1), df.drop(labels=['target'], axis=1), df['target'], cv=10, scoring='f1')

display(scr_f1)
display(np.mean(scr_f1))
```

Vai dar problema se uma questao pede score no **treino**. Um jeito mais geral de fazer o cross-validation esta escrito abaixo:

In [0]:
from sklearn.svm             import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics         import f1_score

df = pd.read_csv('{}/itub/Questoes/classificacao_1.csv'.format(PATH))
df.head()

scores = list()

X = df.drop(labels=['target'], inplace=False, axis=1)
y = df['target']

for train_id, test_id in KFold(n_splits=10).split(df):
  X_train, X_test = X.iloc[train_id], X.iloc[test_id]
  y_train, y_test = y.iloc[train_id], y.iloc[test_id]

  svc = SVC(kernel='linear', C=1)
  svc.fit(X_train, y_train)
  
  # NAS PARTICOES DE TREINO !
  # y_pred = svc.predict(X_test)
  # scores.append(f1_score(y_test, y_pred))

  y_pred = svc.predict(X_train)
  scores.append(f1_score(y_train, y_pred))

display(scores)
display(np.mean(scores))

### Unsupervised

#### Clustering

##### Hierarchical Clustering

Strategies for hierarchical clustering generally fall into two types:

**Agglomerative**, a "bottom-up" approach: each observation starts in its own cluster, and pairs of clusters are merged as one moves up the hierarchy. **Divisive**, a "top-down" approach: all observations start in one cluster, and splits are performed recursively as one moves down the hierarchy [ref](https://www.kaggle.com/vipulgandhi/hierarchical-clustering-explanation).

Agglomerative clustering possuem algumas formas diferentes de medir dissimilaridade entre grupos, e isso define a variante do metodo.<br/>
**single link** onde a distancia minima entre os membros de cada par de grupos e a medida similaridade; capaz de separar coisas nao-elipticas, mas fragil com respeito a ruido entre clusters.<br/>
**complete link** em que a distancia maxima entre os membros e a medida de dissimilaridade; lida bem com ruido, mas ha um vies globular nessa abordagem, alem da tendencia de quebrar grandes clusters.<br/>
**average link** em que a distancias entre os centroides e a medida de dissimilaridade; lida bem com ruido, mas tambem possui vies globular [ref](https://towardsdatascience.com/understanding-the-concept-of-hierarchical-clustering-technique-c6e8243758ec).

Do Divisive clustering surge o **bisecting K-Means**, em que sucessivos 2-Means sao aplicados em cada cluster maior.


##### K-means

[ref](https://www.kaggle.com/vipulgandhi/kmeans-detailed-explanation) com detalhes.

* K-Medoids

Em geral K-means usa a squared euclidian distance -- provavelmente para evitar o calculo da raiz quadrada. Isso torna o K-means menos robusto a outliers. A proposta do K-Medoids e corrigir isso. A ideia central e encontrar objetos representativos no conjunto de dados [ref](https://www.youtube.com/watch?v=GApaAnGx3Fw). Uma vez definidos os objetos representativos -- medoids -- a atualizacao acontece pela verificacao se algum outro objeto diminui a soma de medidas de similaridade. E mais caro que o K-Means, porem mais robusto.

* K-Means++

Inicializacoes ruins geram agrupamentos ruins no K-means -- alem de eventualmente aumentar o tempo necessario para convergencia --, entao o K-Means++ tenta corrigir isso, gerando inicializacoes quasi-otimas. A ideia consiste em escolher o primeiro centroide aleatoriamente entre os pontos a serem agrupados e a partir do segundo centroide o posicionamento e feito aleatoriamente de acordo com a probabilidade proporcional ao quadrado da distancia dos centroides existentes. Isso garante que **os centroides sejam inicializados de modo mais distante entre si**. [ref](https://medium.com/machine-learning-algorithms-from-scratch/k-means-clustering-from-scratch-in-python-1675d38eee42)

Com respeito a outliers, qualquer metodo de inicializacao e otimizacao vai ser sensivel a outliers dado que a funcao objetivo e a soma de quadrados.

* Mini Batch K-Means

Mais rapido.

* Streaming K-Means

Versao online do K-means -- e portanto analoga ao Mini Batch K-Means -- em que novos dados vao surgindo e a partir dos quais a posicao dos clusteres vai sendo atualizadas. Eventualmente e possivel usar hiper-parametros para diminuir a importancia de dados muito antigos alem de ser possivel remover clusteres nao atualizados ha muito tempo. [ref](https://stats.stackexchange.com/questions/222235/explain-streaming-k-means)

* K-means vs K-medians

Usa mediana no lugar da raiz da distancia euclidiana, entao troca uma metrica $\ell^2$ por uma $\ell$.

* Elbow vs Silhoutte

**Elbow**: Inertia $\times$ k-esimo cluster.<br/> 
**Silhouette**: The silhouette value measures how similar a point is to its own cluster (cohesion) compared to other clusters (separation) [ref](https://medium.com/analytics-vidhya/how-to-determine-the-optimal-k-for-k-means-708505d204eb). 

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/3d80ab22fb291b347b2d9dc3cc7cd614f6b15479)

onde $a(i)$ e a media entre i e todos os pontos, representando quao bem o i-esimo ponto esta associado **ao seu cluster**; $b(i)$ e a menor distancia entre e todos os outros pontos **dos outros clusters**, representando a dissimilaridade.

Em particular para o caso de GMM a BIC pode ser uma metrica mais adequada para escolha do numero otimo de clusters.

* Inertia vs Distortion

**Inertia** e a soma das distancias quadraticas entre as amostras o cluster mais proximo. **Distortion** e a media das distancias quadraticas dos centroides aos elementos associados a eles.


- - -

Snippet:

In [0]:
from sklearn.cluster import KMeans, MiniBatchKMeans

agr      = pd.read_csv('{}/itub/Questoes/agrupamento.csv'.format(PATH))
centroid = pd.read_csv('{}/itub/Questoes/centroides_iniciais.csv'.format(PATH))

display(agr.head())
display(agr.shape)
display(centroid.head())

# elbow method setting cluster centroids

inertia = list()
for k in range(1, 16):
  kmeans = KMeans(n_clusters = k)
  kmeans.cluster_centers_ = centroid.iloc[:k ,:].as_matrix()
  kmeans.fit(agr)
  inertia.append(kmeans.inertia_)

plt.plot([i for i in range(1,16)], inertia, '--.')

# listing inertia

for i in range(1,16):
  print(i, inertia[i-1])

##### Mean Shift

[ref](https://dashee87.github.io/data%20science/general/Clustering-with-Scikit-with-GIFs/), [ref](http://efavdb.com/mean-shift/) com mais detalhes sobre a regra de atualizacao dos pontos, e [ref](https://spin.atomicobject.com/2015/05/26/mean-shift-clustering/) com aplicacao em image segmentation.

#### Pattern Search

#### Dimension Reduction

## 2. Ensemble Methods

Em geral ensemble learning e a aprendizagem a partir da forma de uma combinacao linear de modelos base 

$$f(y|x,\pi) = \sum_{m \in \mathcal{M}} w_m f_m(y|x)$$

Observacoes:
1. ha uma relacao funcional bem clara entre ensemble learning e adaptive basis-function learning, portanto **boosting tambem podem ser pensandos como ensemble learning** em que os pesos sao determinados sequencialmente, uma vez que boosting podem ser pensados como uma forma de fittar adaptives basis-function models de forma greedy. 

$$f(x) = w_0 + \sum_m^M w_m \phi_m (x)$$

2. ha uma relacao aparente entre ensemble learning e **neural networks**, em que $f_m$ e analogo a m-esima camada oculta e $w_m$ sao os pesos das camadas output.


### Boosting

### Stacking

### Bagging

## 3. Reinforcement Learning

## 4. Deep Learning

- - -

# In-Depth Topics

## Learning Paradigms

Beyond Supervised vs Unsupervised.

**Batch vs Online Learning**

_Batch learning_ é o aprendizado feito de uma vez com todo o conjunto de dados disponível, de modo que se algo precisa ser atualizado -- um novo tipo de spam -- é necessário treinar tudo novamente, novo spam jutamente com todos os antigos treinados anteriormente.

_Online learning_ é o aprendizado incrimental, observação a observação ou através de mini-batches. 

- - -

**Instance-Based vs Model-Based Learning**

_Instance-Based_ generaliza novos casos a partir dos casos aprendidos, através de alguma métrica de similaridade.

_Model-Based_ generaliza novos casos através dos parâmetros de um modelo construído a priori.

## Bias-Variance Tradeoff


## Outliers

## Validation

Dado um conjunto de dados $\mathcal{D}$

- $\mathcal{T}$ treino (0.8 * $\mathcal{D}$)
- $\mathcal{V}$ validação, ou _holdout validation_ (0.2 * $\mathcal{T}$)
- $\mathcal{T'}$ teste (0.2 * $\mathcal{D}$)

$\mathcal{V}$ pode ser usado para avaliar vários modelos candidatos. O fluxo então é

1. $M_1, M_2, M_3 \rightarrow \mathcal{T}$
2. $M_1, M_2, M_3 \rightarrow \mathcal{V} \Rightarrow M^{*}$ 
3. $M^{*} \rightarrow \mathcal{T'}$

### Cross-Validation

Dentre as infinitas formas de definir: modelo complexo $\to$ ruido capturado  $\to$ overfitting.

## Cost vs Loss vs Objective

+ **Loss function** is usually a function **defined on a data point**, prediction and label, and measures the penalty. Ex: square loss @ linear regression.

+ **Cost function** is usually more general. It might be a **sum of loss functions over your training** set plus some model complexity penalty (regularization). Ex: mse @ linear regression

+ **Objective function** is the most general term for **any function that you optimize during training**. Ex: mle @ linear regression 

[ref](https://stats.stackexchange.com/questions/179026/objective-function-cost-function-loss-function-are-they-the-same-thing)

## Distance Metrics

![](https://raw.githubusercontent.com/thiagodsd/thiagodsd.github.io/master/img/index.png)

+ Chebyshev
+ [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance)
+ [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

## Performance Metrics

Identificando, modelo $h$ tal que $h(x) = \hat{y}$ e a predicao para $y$ dado $x$.

### Regression

* MAE, mean absolute error $$\frac{1}{N}\sum_i^N |y_i - h(x)|$$

* MSE, mean squared error $$\frac{1}{N}\sum_i^N (y_i - h(x))^2$$

* RMSE, root mean squared error $$\sqrt{\frac{1}{N}\sum_i^N (y_i - h(x))^2}=\sqrt{\text{MSE}}$$

* $R^2$, coefficient of determination $$1 - \frac{\sum (y_i - h(x))^2}{\sum(y_i  - \bar{y})^2}$$

* $\bar{R}^2$, adjusted coefficient of determination, para $k$ features e $n$ linhas $$1 - (1-R^2)\frac{n-1}{n-(k+1)}$$

- - -

Como sempre -- devido $\ell_1$ vs $\ell_2$ -- MAE melhor que RMSE caso o conjunto tenha muitos outliers. Por outro lado MAE envolve calculo de modulo, que geralmente e caro.

Particularidade da RMSE: ela e sensivel a variancia da distribuicao de magnitude dos erros -- o que e diferente de ser sensivel a variancia dos erros. Portanto RMSE **nao mede apenas o erro**.

### Classification